# MiniProyecto 4: Optimización de parámetros

¡Bienvenidos al cuarto (mini)proyecto de la carrera de Data Science de Acamica! 

En este proyecto vamos a seguir trabajando (por última vez) con el dataset de propiedades en venta publicadas en el portal [Properati](www.properati.com.ar). El objetivo en este caso es optimizar los parámetros de los algoritmos que usamos en el proyecto pasado.

El dataset es el mismo del proyecto 3. Recordemos que las columnas que se agregan son:

* `barrios_match`: si coincide el barrio publicado con el geográfico vale 1, si no 0.

* `PH`, `apartment`, `house`: variables binarias que indican el tipo de propiedad.

* dummies de barrios: variables binarias con 1 o 0 según el barrio.

La métrica que vamos a usar para medir es RMSE (raíz del error cuadréatico medio), cuya fórmula es:

$$RMSE = \sqrt{\frac{\sum_{t=1}^n (\hat y_t - y_t)^2}{n}}$$

## Pandas - Levantamos el dataset

In [6]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
path_dataset = r'C:\Users\causa\3) Machine Learning\entrega+4\dataset\datos_properati_limpios_model.csv'
df = pd.read_csv(path_dataset)

**Separá** el dataset en entrenamiento (80%) y test (20%) utilizando como target la columna `price_aprox_usd`

In [ ]:
# Hacé la separación en esta celda

In [7]:
# Esto es lo que hace
import numpy as np
np.random.seed(123)
from sklearn.model_selection import train_test_split
X = df.drop(['price_aprox_usd'], axis=1)
y = df['price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape[0], X_test.shape[0])

5100 1276


## Scikit-learn - Entrenamiento

Para repasar los parámetros de árboles de decisión en Scikit-learn: 

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

En primer lugar veamos como hacer cross validation. Para eso necesitamos definir la cantidad de folds, en este caso vamos a usar 5.

GridSearchCV nos permite testear a través de un espacio de búsqueda de parámetros la mejor combinación posible dado un estimador.

Por ejemplo, en este caso probamos la profundidad máxima y la máxima cantidad de features para hacer los split. Ambos entre 1 y 5.
Recordemos que para hacer la optimización scikit-learn usa la métrica `neg_mean_squared_error` en lugar de `mean_squared_error`.

**Creá** una variable `param_grid` con valores del 1 al 5 para los atributos `max_depth` y `max_features`. 

In [8]:
# Creá en esta celda la variable param_grid
param_grid = {'max_depth':list(range(1,6)), 'max_features':list(range(1,6))}
param_grid

{'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]}

**Importá** `GridSearchCV` y `DecisionTreeRegressor`.

**Creá** una variable `grid_search` y asignale un `GridSearchCV` que recorra el `param_grid` que creaste con el algoritmos `DecisionTreeRegressor` y el un scoring de `neg_mean_squared_error`

In [9]:
# Importa y crea un GridSearchCV en esta celda
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
dtr = DecisionTreeRegressor()
grid_search = GridSearchCV(dtr, param_grid = param_grid, scoring = 'neg_mean_squared_error', cv = 10, return_train_score = True)

A continuación, realizá el `fit` del `grid_search` con el conjunto de entrenamiento

In [10]:
# Hace el fit de grid search en esta celda
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [1, 2, 3, 4, 5], 'max_features': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Revisemos los resultados. Recordemos que no están expresados en RMSE.

In [44]:
import pandas as pd
results = pd.DataFrame(grid_search.cv_results_)[['param_max_depth', 'param_max_features','mean_test_score', 'std_test_score']]
results.describe()

,mean_test_score,std_test_score
count,25.000,25.000
mean,-896358188.719,87779122.809
std,66824617.546,29651866.319
min,-976750859.061,37523655.597
25%,-940036217.281,74435662.971
50%,-931032807.533,86895165.336
75%,-855343674.498,104792128.211
max,-750289561.448,150024745.348


**Mostrá** los `grid_scores` obtenidos durante el `grid_search`

In [45]:
# Mostrá los grid_scores en esta celda
results

,param_max_depth,param_max_features,mean_test_score,std_test_score
0,1,1,-970979036.406,48551403.676
1,1,2,-976750859.061,37523655.597
2,1,3,-950696027.296,83945351.686
3,1,4,-892492051.781,137599517.283
4,1,5,-934985212.235,74435662.971
5,2,1,-945552769.215,92773735.668
6,2,2,-970342666.897,37571853.371
7,2,3,-940036217.281,79450580.636
8,2,4,-939946731.001,55262721.176
9,2,5,-939346953.995,44518352.948


De esta manera, el valor con mejor resultado (dado el espacio de búsqueda definido) es `max_depth` 3 y `max_features` 3.

**Mostrá** el mejor score y los mejores parámetros encontrados por `grid_search`

In [11]:
# Mostrás los resultados en esta celda
grid_search.best_score_, grid_search.best_params_

(-750289561.4480243, {'max_depth': 5, 'max_features': 5})

In [53]:
score = nmsq2rmse(grid_search.cv_results_['mean_test_score'])
pd.DataFrame(score, columns = ['RMSE'])

,RMSE
0,31160.540
1,31253.010
2,30833.360
3,29874.610
4,30577.530
5,30749.840
6,31150.320
7,30660.010
8,30658.550
9,30648.770


Convertimos a RMSE.

In [48]:
def nmsq2rmse(score):
    return np.round(np.sqrt(-score), 2)

__Encontrar el mejor modelo para el espacio de búsqueda dado__

* `"min_samples_split": [2, 10, 20]`
* `"max_depth": [None, 2, 5, 10, 15]`
* `"min_samples_leaf": [1, 5, 10, 15]`
* `"max_leaf_nodes": [None, 5, 10, 20]`

In [17]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
param_grid = {"min_samples_split": [2, 10, 20],
             "max_depth": [None, 2, 5, 10, 15],
             "min_samples_leaf": [1, 5, 10, 15],
             "max_leaf_nodes": [None, 5, 10, 20]}
grid_search = GridSearchCV(regressor, param_grid = param_grid, scoring = 'neg_mean_squared_error', cv = 10,
                          return_train_score = True,
                          refit = True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_samples_split': [2, 10, 20], 'max_depth': [None, 2, 5, 10, 15], 'min_samples_leaf': [1, 5, 10, 15], 'max_leaf_nodes': [None, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Recordemos que `GridSearchCV` tiene como parámetro default `refit=True`. Esto significa que luego de hacer la corrida se ajusta el mejor modelo al conjunto de datos de entrada. De esta manera, se puede predecir directamente usando `best_estimator_`.

In [18]:
optimised_decision_tree = grid_search.best_estimator_

__Evaluemos en testing el desempeño de este modelo.__

Como venimos trabajando, el resultado en testing será la medición que usaremos como benchmark para comparar este modelos con otros en el futuro, puesto que no estuvo en contacto con el dataset de test para la calibración.

In [19]:
from sklearn.metrics import mean_squared_error
y_opt_pred = optimised_decision_tree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_opt_pred))
np.round(rmse)

21301.0

Vemos los primeros 10 resultados de la predicción del valor de propiedades.

In [20]:
val_real = pd.Series(y_test.values)
val_pred = pd.Series(y_opt_pred)

In [21]:
predicciones = pd.concat([val_real.rename('Valor real'),val_pred.rename('Valor Pred') ,abs(val_real-val_pred).rename('Dif(+/-)')] ,  axis=1)

In [22]:
predicciones.head(10)

,Valor real,Valor Pred,Dif(+/-)
0,80000.000,103438.660,23438.660
1,128000.000,135705.882,7705.882
2,150000.000,156075.759,6075.759
3,85000.000,102400.991,17400.991
4,135000.000,135571.622,571.622
5,135000.000,109560.000,25440.000
6,68000.000,75181.250,7181.250
7,110000.000,140444.444,30444.444
8,134000.000,158431.250,24431.250
9,110000.000,76701.202,33298.798
